In [ ]:
import warnings
import torch
import gradio
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# **1.display the orginal text by link**



In [ ]:
summarizer = pipeline("summarization", model="stevhliu/my_awesome_billsum_model")

In [ ]:
url = "https://medium.com/analytics-vidhya/openai-gpt-3-language-models-are-few-shot-learners-82531b3d3122"

In [ ]:
r = requests.get(url)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['hl', 'p'])
text = [result.text for result in results]
Article = ''.join(text)

In [ ]:
Article

'Sign upSign inSign upSign inSoheil TehranipourFollowAnalytics Vidhya--4ListenShareOpenAI recently published a paper describing GPT-3, a deep-learning model for Natural Language Processing, with 175 Billion parameters(!!!), 100x more than the previous version, GPT-2. The model is pre-trained on nearly half a trillion words and achieves SOTA performance on several NLP benchmarks without fine-tuning.Based on its paper, GPT-3 is an autoregressive language model as opposed to a denoising autoencoder like BERT. I decided to write about some of the comparative differences between those two architectures of language models. The paper is an investigation into what you can do with giant language models. Now this language model is an order of magnitude larger than anyone has ever built a language model.OpenAI’s researchers: “In this work we do not fine-tune GPT-3 because our focus is on task-agnostic performance,”So, they seem appropriate for natural language generation. One of the things I want

# **2.Separate into different parts**

In [ ]:
max_chunk = 500

In [ ]:
Article = Article.replace('.', '')
Article = Article.replace('?', '')
Article = Article.replace('!', '')

In [ ]:
sentences = Article.split(' ')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
len(chunks)

2

In [ ]:
chunks[0]

'Sign upSign inSign upSign inSoheil TehranipourFollowAnalytics Vidhya--4ListenShareOpenAI recently published a paper describing GPT-3, a deep-learning model for Natural Language Processing, with 175 Billion parameters(), 100x more than the previous version, GPT-2 The model is pre-trained on nearly half a trillion words and achieves SOTA performance on several NLP benchmarks without fine-tuningBased on its paper, GPT-3 is an autoregressive language model as opposed to a denoising autoencoder like BERT I decided to write about some of the comparative differences between those two architectures of language models The paper is an investigation into what you can do with giant language models Now this language model is an order of magnitude larger than anyone has ever built a language modelOpenAI’s researchers: “In this work we do not fine-tune GPT-3 because our focus is on task-agnostic performance,”So, they seem appropriate for natural language generation One of the things I want to explor

# **3.Summarizing each part**

In [ ]:
res = summarizer(chunks, max_length=50, min_length=30, do_sample=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (700 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
len(res)

2

In [ ]:
res[1]

{'summary_text': 'GPT-3 is a direct scale-up of GPT, with 175 billion parameters and trained on more than 10X the amount of data (Based on OpenAI) It reflects substantial performance on various NLP tasks and benchmarks'}

In [ ]:
for i in range(len(res)):
    print(res[i])

{'summary_text': 'GPT-3 is a deep-learning model for Natural Language Processing, with 175 Billion parameters(), 100x more than the previous version, GPT-2 The model is pre-trained on nearly half a trillion words and'}
{'summary_text': 'GPT-3 is a direct scale-up of GPT, with 175 billion parameters and trained on more than 10X the amount of data (Based on OpenAI) It reflects substantial performance on various NLP tasks and benchmarks'}


# *** Display in app**

In [ ]:
def SUMMARIZE(Url):
    summarizer = pipeline("summarization", model="stevhliu/my_awesome_billsum_model")
    r = requests.get(Url)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all(['hl', 'p'])
    text = [result.text for result in results]
    Article = ''.join(text)
    sentences = Article.split(' ')
    current_chunk = 0
    chunks = []
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            #print(current_chunk)
            chunks.append(sentence.split(' '))

    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])
    res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)
    for i in range(len(res)):
        return res[i].values()

**get the link, show the suumarized text**

In [ ]:
interface = gradio.Interface(fn=SUMMARIZE,
                             inputs=gradio.TextArea(lines=2, placeholder='Enter your url'),
                             outputs=gradio.TextArea())
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
